In [1]:
# Installs
!pip install flask pyngrok

# Imports
from flask import Flask, request, jsonify
from pyngrok import ngrok
import torch  # Or the framework your model uses
import os
import time
import json
import regex as re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel, PeftConfig


# I dunno if kaggle has an .env
ngrok.set_auth_token('2wjAHHT3hF4pB8QYXfCMy5cNMyJ_7qBbJPRrWxswuCcsz4Fjg')

2025-05-06 18:32:30.079488: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746556350.531790      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746556350.670013      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def clean_arabic(text):
    # Remove newlines
    text = re.sub(r'\\n|\n', ' ', text)

    # Normalize 'أ', 'إ', 'آ' -> 'ا'
    text = re.sub(r'[أإآ]', 'ا', text)

    # Normalize 'ة' -> 'ه'
    text = re.sub(r'[ة]', 'ه', text)

    # Normalize Eastern Arabic numerals to Western Arabic numerals
    arabic_numerals = '٠١٢٣٤٥٦٧٨٩'
    western_numerals = '0123456789'
    trans = str.maketrans(arabic_numerals, western_numerals)
    text = text.translate(trans)

    # Remove diacritics (tashkeel)
    text = re.sub(r'[\u064B-\u065F]', '', text)

    # Remove tatweel
    text = re.sub(r'\u0640', '', text)

    # Remove Arabic and Western punctuation
    text = re.sub(r'[،؛«»,!?()\[\]{}"\'\\]', '', text)

    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

# Example usage
example_text = "أهلًا وسهلًا! كيف حالك؟"
cleaned_text = clean_arabic(example_text)
print(cleaned_text)


اهلا وسهلا كيف حالك؟


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel, PeftConfig

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("ZiadWaleed/mBart-LoRA-for-MT-ar-en2")

# Load PEFT config to get base model name
config = PeftConfig.from_pretrained("ZiadWaleed/mBart-LoRA-for-MT-ar-en2")
base_model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)

# Load LoRA adapter on top of base model
model = PeftModel.from_pretrained(base_model, "ZiadWaleed/mBart-LoRA-for-MT-ar-en2")


# Set model to evaluation mode
model.eval()

# Set up for translation
SRC_LANG = "ar_AR"
TGT_LANG = "en_XX"

tokenizer.src_lang = SRC_LANG
tokenizer.tgt_lang = TGT_LANG

# Set forced BOS token to the target language
model.config.forced_bos_token_id = tokenizer.lang_code_to_id[TGT_LANG]

tokenizer_config.json:   0%|          | 0.00/11.0k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/813 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['corda_config', 'trainable_token_indices'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

In [4]:
def translate(arabic_text , model):

    device = torch.device("cpu")
    model = model.to(device)
    arabic_text = clean_arabic(arabic_text)
    inputs = tokenizer(arabic_text, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to("cpu") for k, v in inputs.items()}  # Ensure on CPU

    model.eval()
    with torch.no_grad():
        translated_tokens = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=128,
            num_beams=5,
            early_stopping=True,
        )

    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    return translated_text


In [5]:
app = Flask(__name__)

@app.route('/api/predict', methods=['POST'])
def predict():
    """
    Expects JSON with 'text' field
    """
    data = request.json
    
    if not data or 'text' not in data:
        return jsonify({
            'error': 'Missing required field: text'
        }), 400
    
    if re.fullmatch('^[A-Za-z]+$', data['text']):
        return jsonify({
            'error': "Invalid Language - English text detected"
        }), 400

    text = clean_arabic(data['text'])
    
    try:
        result = translate(text, model);
        return jsonify(result)
    
    except Exception as e:
        return jsonify({
            'error': str(e)
        }), 500

@app.route('/api/status', methods=['GET'])
def status():
    return jsonify({
        'status': 'up',
        'model': 'loaded',
        'timestamp': time.strftime("%Y-%m-%d %H:%M:%S")
    })

@app.after_request
def after_request(response):
    response.headers.add('Access-Control-Allow-Origin', '*')
    response.headers.add('Access-Control-Allow-Headers', 'Content-Type,Authorization')
    response.headers.add('Access-Control-Allow-Methods', 'GET,POST,OPTIONS')
    return response

In [ ]:
def start_server():
    port = 5000
    public_url = ngrok.connect(port).public_url
    print(f" * Ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")
    print(f" * API endpoint: {public_url}/api/predict")
    app.run(host='0.0.0.0', port=port)

if __name__ == '__main__':
    start_server()


 * Ngrok tunnel "https://ca46-34-28-237-149.ngrok-free.app" -> "http://127.0.0.1:5000"
 * API endpoint: https://ca46-34-28-237-149.ngrok-free.app/api/predict
 * Serving Flask app '__main__'
 * Debug mode: off


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1666: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
